### Installation 

In [ ]:
! pip install datasets transformers rouge-score nltk
!git clone https://github.com/xmu-xiaoma666/External-Attention-pytorch.git ext_attns

### About dataset 

In [4]:
print(f'train: {len(tr)}')
print(f'val:   {len(val)}')
print(f'test:  {len(te)}')

train: 1020
val:   408
test:  409


### 2L Attentions

|i|attention | used|
|---|--------| ----|
|0  |UFOAttention| ✔|
|1  | AFT_FULL   |x|
|2  | MUSEAttention|✔ |
|3  | EMSA|     x|
|4  |SimplifiedScaledDotProductAttention|✔ |
|5  |ScaledDotProductAttention|✔ |
|6  |ExternalAttention|✔ |

In [1]:
import os
ext_path = os.path.abspath('./ext_attns/')
import sys
sys.path.append(ext_path)
# list
from model.attention.UFOAttention import *
from model.attention.AFT import AFT_FULL
from model.attention.MUSEAttention import MUSEAttention
from model.attention.EMSA import EMSA
from model.attention.SimplifiedSelfAttention import SimplifiedScaledDotProductAttention
from model.attention.SelfAttention import ScaledDotProductAttention
from model.attention.ExternalAttention import ExternalAttention
d_model = 768 
n_head  = 12  
ufo = UFOAttention(d_model=d_model, d_k=d_model//n_head, d_v=d_model//n_head, h=n_head)
aft_full = AFT_FULL(d_model=d_model, n=n_head)
ma = MUSEAttention(d_model=d_model, d_k=d_model//n_head, d_v=d_model//n_head, h=n_head)
emsa = EMSA(d_model=d_model, d_k=d_model//n_head, d_v=d_model//n_head, h=n_head,H=n_head,W=n_head,ratio=2,apply_transform=True)
ssa = SimplifiedScaledDotProductAttention(d_model=d_model, h=n_head)
sa = ScaledDotProductAttention(d_model=d_model, d_k=d_model//n_head, d_v=d_model//n_head, h=n_head)
ea = ExternalAttention(d_model=d_model,S=8) #v

attn_li = [ufo,aft_full,ma,emsa,ssa,sa,ea]

import transformers

# print(transformers.__version__)

model_checkpoint = "t5-small"
model_checkpoint = 'facebook/bart-base'

from datasets import load_dataset, load_metric

raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")

# raw_datasets["train"][0]

import datasets

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# ?
# with tokenizer.as_target_tokenizer():
#     print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

batch_size = 3 #16

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

trvaltest = tokenized_datasets['train'].train_test_split(test_size=0.004,train_size=0.005) # 800,1080
# trvaltest = tokenized_datasets['train'].train_test_split(test_size=0.004,train_size=0.01) # 800,2080
# trvaltest = tokenized_datasets['train'].train_test_split(test_size=0.004,train_size=0.02) # 800,4080
tr = trvaltest['train']
valtest = trvaltest['test'].train_test_split(test_size=0.5,train_size=0.5) # 208, 208
# valtest = trvaltest['test'].train_test_split(test_size=0.5,train_size=0.5) # 408, 408
# valtest = trvaltest['test'].train_test_split(test_size=0.05,train_size=0.05)
val = valtest['train']
te = valtest['test']

del trvaltest
del valtest

Using custom data configuration default
Reusing dataset xsum (/home/yp/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/yp/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-5d732b1c86657ea0.arrow
Loading cached processed dataset at /home/yp/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-38966a651c72a103.arrow
Loading cached processed dataset at /home/yp/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-d19c279816291ade.arrow
Loading cached split indices for dataset at /home/yp/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-a37fd9f11dd44add.arrow and /home/yp/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-195107633ef23937.arrow


In [5]:
import copy
from torch import nn
evals = {}
for i,attn in enumerate(attn_li[:]):
    not_allowed = ('AFT_FULL','EMSA')
    print(attn)
    if attn.__str__().startswith(not_allowed):
        continue
    
    import os
    model_name = attn.__str__().split('(')[0]
    # model_name = model_checkpoint.split("/")[-1] + model_name
    saved_path = os.path.join('./output',model_name)
    
    args = Seq2SeqTrainingArguments(
        saved_path,
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=15,
        predict_with_generate=True,
        fp16=True,
        save_strategy='epoch',
        load_best_model_at_end=True,
    #     push_to_hub=True,
    )

    class tel2lin1(nn.Module):
        def __init__(self):
            super(tel2lin1,self).__init__()
            # self.tel = nn.TransformerEncoderLayer(d_model=768,nhead=3,batch_first=True)
            tels = [copy.deepcopy(attn) for _ in range(2) ] 
            self.tels = nn.ModuleList(tels)
            self.lin = nn.Linear(in_features=768,out_features=50265, bias=False)
            
        def forward(self,x):
            if attn.__str__().startswith('ExternalAttention'):
                for tel in self.tels:
                    x = tel(x)
            else:
                for tel in self.tels:
                    x = tel(x,x,x)
            x = self.lin(x)
            return x
    model.lm_head = tel2lin1()

    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tr,
        eval_dataset=val,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    
    evals[f'{model_name}'] = trainer.evaluate(eval_dataset=te)

UFOAttention(
  (fc_q): Linear(in_features=768, out_features=768, bias=True)
  (fc_k): Linear(in_features=768, out_features=768, bias=True)
  (fc_v): Linear(in_features=768, out_features=768, bias=True)
  (fc_o): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, document, summary.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 2550
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a no

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,7.476961,0.062200,0.000000,0.056200,0.056200,4.426500
2,No log,7.080842,14.674200,0.160500,12.896500,12.891400,14.115200
3,7.462100,6.937696,13.002700,0.954200,12.178900,12.145300,14.247500
4,7.462100,6.844901,17.655000,1.336400,15.069400,15.213800,20.000000
5,7.462100,6.780678,13.566600,0.801100,11.531400,11.689400,20.000000
6,6.169400,6.785341,13.654600,0.312100,11.405600,11.423500,19.985300
7,6.169400,6.756629,15.945100,0.626100,13.097100,13.322200,19.958300
8,6.169400,6.754077,13.645900,0.194900,10.882300,11.000800,20.000000
9,5.837300,6.713006,15.595000,0.481300,12.261900,12.311300,20.000000
10,5.837300,6.735507,15.973300,0.396200,12.838900,12.959000,20.000000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, document, summary.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 6
Saving model checkpoint to ./output/UFOAttention/checkpoint-170
Configuration saved in ./output/UFOAttention/checkpoint-170/config.json
Model weights saved in ./output/UFOAttention/checkpoint-170/pytorch_model.bin
tokenizer config file saved in ./output/UFOAttention/checkpoint-170/tokenizer_config.json
Special tokens file saved in ./output/UFOAttention/checkpoint-170/special_tokens_map.json
Deleting older checkpoint [output/UFOAttention/checkpoint-2040] due to args.save_total_limit
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimens

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


ExternalAttention(
  (mk): Linear(in_features=768, out_features=8, bias=False)
  (mv): Linear(in_features=8, out_features=768, bias=False)
  (softmax): Softmax(dim=1)
)


Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, document, summary.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 2550


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,10.815591,0.000000,0.000000,0.000000,0.000000,8.000000
2,No log,10.773495,0.000000,0.000000,0.000000,0.000000,5.000000
3,10.784300,10.694166,0.000000,0.000000,0.000000,0.000000,5.000000
4,10.784300,10.591418,0.000000,0.000000,0.000000,0.000000,5.000000
5,10.784300,10.477371,0.000000,0.000000,0.000000,0.000000,5.000000
6,10.512700,10.360878,0.000000,0.000000,0.000000,0.000000,5.000000
7,10.512700,10.248258,0.000000,0.000000,0.000000,0.000000,5.000000
8,10.512700,10.144051,0.000000,0.000000,0.000000,0.000000,5.000000
9,10.142200,10.051143,0.000000,0.000000,0.000000,0.000000,5.000000
10,10.142200,9.971609,0.000000,0.000000,0.000000,0.000000,5.000000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, document, summary.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 6
Saving model checkpoint to ./output/ExternalAttention/checkpoint-170
Configuration saved in ./output/ExternalAttention/checkpoint-170/config.json
Model weights saved in ./output/ExternalAttention/checkpoint-170/pytorch_model.bin
tokenizer config file saved in ./output/ExternalAttention/checkpoint-170/tokenizer_config.json
Special tokens file saved in ./output/ExternalAttention/checkpoint-170/special_tokens_map.json
Deleting older checkpoint [output/ExternalAttention/checkpoint-2210] due to args.save_total_limit
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Wa

### Light 2L Attentions 

### 2L Attentions with Skip connection 

#### 1 step 

In [2]:
import copy
from torch import nn
evals = {}
for i,attn in enumerate(attn_li[:]):
    not_allowed = ('AFT_FULL','EMSA')
    print(attn)
    if attn.__str__().startswith(not_allowed):
        continue
    
    import os
    model_name = attn.__str__().split('(')[0]
    # model_name = model_checkpoint.split("/")[-1] + model_name
    saved_path = os.path.join('./output',model_name)
    
    args = Seq2SeqTrainingArguments(
        saved_path,
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=15,
        predict_with_generate=True,
        fp16=True,
        save_strategy='epoch',
        load_best_model_at_end=True,
    #     push_to_hub=True,
    )

    class tel2lin1(nn.Module):
        def __init__(self):
            super(tel2lin1,self).__init__()
            # self.tel = nn.TransformerEncoderLayer(d_model=768,nhead=3,batch_first=True)
            tels = [copy.deepcopy(attn) for _ in range(2) ] 
            self.tels = nn.ModuleList(tels)
            self.lin = nn.Linear(in_features=768,out_features=50265, bias=False)
            
        def forward(self,x):
            if attn.__str__().startswith('ExternalAttention'):
                for tel in self.tels:
                    x = tel(x)+x
            else:
                for tel in self.tels:
                    x = tel(x,x,x)+x
            x = self.lin(x)
            return x
    model.lm_head = tel2lin1()

    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tr,
        eval_dataset=val,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    
    evals[f'{model_name}'] = trainer.evaluate(eval_dataset=te)

UFOAttention(
  (fc_q): Linear(in_features=768, out_features=768, bias=True)
  (fc_k): Linear(in_features=768, out_features=768, bias=True)
  (fc_v): Linear(in_features=768, out_features=768, bias=True)
  (fc_o): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 2550
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,6.795927,17.742100,1.650300,15.602800,15.602900,19.958300
2,No log,6.359220,16.994100,2.412100,13.840100,13.870000,19.745100
3,6.621900,6.208702,17.668900,2.722300,13.836600,13.846900,19.551500
4,6.621900,6.074422,17.759300,2.850600,14.315800,14.280400,19.774500
5,6.621900,5.960795,18.342500,3.174600,14.444600,14.437800,19.951000
6,5.052400,5.916854,18.716700,3.211500,14.940300,14.920300,19.943600
7,5.052400,5.832363,18.119800,3.147100,14.492100,14.437200,19.985300
8,5.052400,5.812722,18.068100,3.353500,14.344100,14.311300,19.990200
9,4.356800,5.749690,17.444700,2.984700,13.887000,13.832300,19.977900
10,4.356800,5.750962,17.878700,3.075600,14.220800,14.166600,19.973000


/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 6
Saving model checkpoint to ./output/UFOAttention/checkpoint-170
Configuration saved in ./output/UFOAttention/checkpoint-170/config.json
Model weights saved in ./output/UFOAttention/checkpoint-170/pytorch_model.bin
tokenizer config file saved in ./output/UFOAttention/checkpoint-170/tokenizer_config.json
Special tokens file saved in ./output/U

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


ExternalAttention(
  (mk): Linear(in_features=768, out_features=8, bias=False)
  (mv): Linear(in_features=8, out_features=768, bias=False)
  (softmax): Softmax(dim=1)
)


Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 2550


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,6.326079,20.660200,3.903400,16.495900,16.461100,19.772100
2,No log,5.896080,20.206900,3.972000,16.225700,16.167300,19.759800
3,3.786900,5.835314,19.847300,4.154400,15.994500,15.989800,19.852900
4,3.786900,5.815123,19.948600,4.165900,16.118900,16.106300,19.818600
5,3.786900,5.815073,20.265300,4.383600,16.577700,16.563100,19.784300
6,2.653900,5.816186,20.279000,4.385500,16.388500,16.338900,19.725500
7,2.653900,5.803371,19.724300,4.240600,16.023400,16.010800,19.799000
8,2.653900,5.830432,20.430800,4.511600,16.624400,16.624000,19.715700
9,2.191400,5.802033,19.926200,4.283100,16.050900,16.048800,19.803900
10,2.191400,5.822743,19.918400,4.475500,16.384400,16.384200,19.732800


/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 6
Saving model checkpoint to ./output/ExternalAttention/checkpoint-170
Configuration saved in ./output/ExternalAttention/checkpoint-170/config.json
Model weights saved in ./output/ExternalAttention/checkpoint-170/pytorch_model.bin
tokenizer config file saved in ./output/ExternalAttention/checkpoint-170/tokenizer_config.json
Special tokens file

#### 1step + 2step 

In [ ]:
import copy
from torch import nn
evals = {}
for i,attn in enumerate(attn_li[:]):
    not_allowed = ('AFT_FULL','EMSA')
    print(attn)
    if attn.__str__().startswith(not_allowed):
        continue
    
    import os
    model_name = attn.__str__().split('(')[0]
    # model_name = model_checkpoint.split("/")[-1] + model_name
    saved_path = os.path.join('./output',model_name)
    
    args = Seq2SeqTrainingArguments(
        saved_path,
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=15,
        predict_with_generate=True,
        fp16=True,
        save_strategy='epoch',
        load_best_model_at_end=True,
    #     push_to_hub=True,
    )

    class tel2lin1(nn.Module):
        def __init__(self):
            super(tel2lin1,self).__init__()
            # self.tel = nn.TransformerEncoderLayer(d_model=768,nhead=3,batch_first=True)
            tels = [copy.deepcopy(attn) for _ in range(2) ] 
            self.tels = nn.ModuleList(tels)
            self.lin = nn.Linear(in_features=768,out_features=50265, bias=False)
            
        def forward(self,x):
            first_x = copy.copy(x)
            if attn.__str__().startswith('ExternalAttention'):
                for tel in self.tels:
                    x = tel(x)+x
            else:
                for tel in self.tels:
                    x = tel(x,x,x)+x
            x = first_x + x
            x = self.lin(x)
            return x
    model.lm_head = tel2lin1()

    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tr,
        eval_dataset=val,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    
    evals[f'{model_name}'] = trainer.evaluate(eval_dataset=te)

### 4L Attentions /w skipConnection

In [2]:
import copy
from torch import nn
evals = {}
for i,attn in enumerate(attn_li[:]):
    not_allowed = ('AFT_FULL','EMSA')
    print(attn)
    if attn.__str__().startswith(not_allowed):
        continue
    
    import os
    model_name = attn.__str__().split('(')[0]
    # model_name = model_checkpoint.split("/")[-1] + model_name
    saved_path = os.path.join('./output',model_name)
    
    args = Seq2SeqTrainingArguments(
        saved_path,
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=15,
        predict_with_generate=True,
        fp16=True,
        save_strategy='epoch',
        load_best_model_at_end=True,
    #     push_to_hub=True,
    )

    class tel2lin1(nn.Module):
        def __init__(self):
            super(tel2lin1,self).__init__()
            # self.tel = nn.TransformerEncoderLayer(d_model=768,nhead=3,batch_first=True)
            tels = [copy.deepcopy(attn) for _ in range(4) ] 
            self.tels = nn.ModuleList(tels)
            self.lin = nn.Linear(in_features=768,out_features=50265, bias=False)
            
        def forward(self,x):
            if attn.__str__().startswith('ExternalAttention'):
                for tel in self.tels:
                    x = tel(x)+x
            else:
                for tel in self.tels:
                    x = tel(x,x,x)+x
            x = self.lin(x)
            return x
    model.lm_head = tel2lin1()

    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tr,
        eval_dataset=val,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    
    evals[f'{model_name}'] = trainer.evaluate(eval_dataset=te)

UFOAttention(
  (fc_q): Linear(in_features=768, out_features=768, bias=True)
  (fc_k): Linear(in_features=768, out_features=768, bias=True)
  (fc_v): Linear(in_features=768, out_features=768, bias=True)
  (fc_o): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 2550
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,6.750159,3.999900,0.498400,3.459500,3.419000,7.102900
2,No log,6.323394,17.025500,2.506100,14.514900,14.517500,19.455900
3,6.522900,6.122281,18.112600,2.869700,14.544300,14.603200,19.808800
4,6.522900,5.994848,17.323300,2.912600,14.146700,14.169000,19.982800
5,6.522900,5.864310,17.452700,2.721100,14.151100,14.157600,19.977900
6,4.849900,5.846464,16.456500,2.659300,13.353000,13.344800,19.924000
7,4.849900,5.733954,16.633700,3.033600,13.551600,13.544400,19.914200
8,4.849900,5.723573,16.561600,2.918000,13.553500,13.583200,19.897100
9,4.037400,5.680391,16.397900,3.148300,13.692500,13.703000,19.980400
10,4.037400,5.651556,16.078300,3.144500,13.290500,13.290100,19.816200


/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 6
Saving model checkpoint to ./output/UFOAttention/checkpoint-170
Configuration saved in ./output/UFOAttention/checkpoint-170/config.json
Model weights saved in ./output/UFOAttention/checkpoint-170/pytorch_model.bin
tokenizer config file saved in ./output/UFOAttention/checkpoint-170/tokenizer_config.json
Special tokens file saved in ./output/U

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


ExternalAttention(
  (mk): Linear(in_features=768, out_features=8, bias=False)
  (mv): Linear(in_features=8, out_features=768, bias=False)
  (softmax): Softmax(dim=1)
)


Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 2550


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,6.253531,20.588800,3.726700,15.990100,16.045500,19.850500
2,No log,5.857163,19.952100,3.996500,15.838500,15.853500,19.759800
3,3.957100,5.812199,19.827600,3.810100,15.602100,15.631500,19.750000
4,3.957100,5.769763,19.588800,3.926900,15.834100,15.856300,19.882400
5,3.957100,5.752831,20.656000,4.055100,16.303400,16.325000,19.902000
6,2.793800,5.768096,20.321300,3.733700,16.034500,16.062000,19.796600
7,2.793800,5.761166,20.239400,4.166600,16.459900,16.458900,19.855400
8,2.793800,5.769475,20.163300,4.018700,16.206200,16.216900,19.823500
9,2.312000,5.745352,20.078600,4.253800,16.243500,16.272000,19.830900
10,2.312000,5.760777,20.595400,4.300500,16.593900,16.579400,19.799000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 6
Saving model checkpoint to ./output/ExternalAttention/checkpoint-170
Configuration saved in ./output/ExternalAttention/checkpoint-170/config.json
Model weights saved in ./output/ExternalAttention/checkpoint-170/pytorch_model.bin
tokenizer config file saved in ./output/ExternalAttention/checkpoint-170/tokenizer_config.json
Special tokens file saved in ./output/ExternalAttention/checkpoint-170/special_tokens_map.json
Deleting older checkpoint [output/ExternalAttention/checkpoint-1530] due to args.save_total_limit
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Wa

### Etc

In [7]:
from torch import nn
import torch,copy
class cons(nn.Module):
    def __init__(self):
        super(cons,self).__init__()
        self.front = nn.Sequential(
            nn.Conv2d(1,32,3),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(32),
            nn.ReLU() ,#batchnorm 2d ?
            nn.Conv2d(32,16,2),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Flatten()
        )
        sple = torch.rand(1,1,28,28)
        front_out = self.front(sple).size()[-1]
        self.body = nn.Sequential(
            nn.Linear(front_out,256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(p=0.2),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            # nn.Linear(128,10),
        )
        self.medium = nn.Sequential(
            nn.Linear(128,64),
            nn.Linear(64,128)
        )
        self.macro = [copy.deepcopy(self.medium) for _ in range(3)]
        self.macro = nn.ModuleList(self.macro)
        
        self.fc = nn.Sequential(
            nn.Linear(128,10),
            nn.Softmax(dim=1)
        )
    def forward(self,x):
        x = self.front(x)
        x = self.body(x)
        # for i in range(len(self.macro)):
        #     x = self.macro[i](x)
        for i in self.macro:
            x = i(x)
        x = self.fc(x)
        return x
md = cons()

In [8]:
from torchinfo import summary as sm
sm(md,(4,1,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
cons                                     --                        --
├─ModuleList: 1-1                        --                        --
├─Sequential: 1-2                        [4, 576]                  --
│    └─Conv2d: 2-1                       [4, 32, 26, 26]           320
│    └─MaxPool2d: 2-2                    [4, 32, 13, 13]           --
│    └─BatchNorm2d: 2-3                  [4, 32, 13, 13]           64
│    └─ReLU: 2-4                         [4, 32, 13, 13]           --
│    └─Conv2d: 2-5                       [4, 16, 12, 12]           2,064
│    └─MaxPool2d: 2-6                    [4, 16, 6, 6]             --
│    └─BatchNorm2d: 2-7                  [4, 16, 6, 6]             32
│    └─ReLU: 2-8                         [4, 16, 6, 6]             --
│    └─Flatten: 2-9                      [4, 576]                  --
├─Sequential: 1-3                        [4, 128]                  --
│    └─Line

In [9]:
md

cons(
  (front): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): Conv2d(32, 16, kernel_size=(2, 2), stride=(1, 1))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Flatten(start_dim=1, end_dim=-1)
  )
  (body): Sequential(
    (0): Linear(in_features=576, out_features=256, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
  )
  (medium): Sequential(
    (

### Eval 

In [4]:
import pickle
with open("Deep4attns_1stepSkip.pickle","wb") as fw:
    pickle.dump(evals, fw)

In [9]:
evals.update(evals_base)

In [7]:
# 2 attn
import pandas as pd
pd.DataFrame(evals).T

,eval_loss,eval_rouge1,eval_rouge2,eval_rougeL,eval_rougeLsum,eval_gen_len,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
UFOAttention,6.666266,14.6945,0.5948,11.9360,11.9617,20.0000,39.8440,10.265,1.732,15.0
MUSEAttention,6.710398,2.7172,0.0000,2.6122,2.6150,11.0660,41.4969,9.856,1.663,15.0
SimplifiedScaledDotProductAttention,7.049309,0.0000,0.0000,0.0000,0.0000,2.0000,39.1412,10.449,1.763,15.0
ScaledDotProductAttention,7.603535,7.1269,0.0000,7.1172,7.0146,11.0293,34.3124,11.920,2.011,15.0
ExternalAttention,9.790882,0.0000,0.0000,0.0000,0.0000,5.0000,29.8197,13.716,2.314,15.0


In [4]:
# 2 attn /w 1step Skip
import pandas as pd
pd.DataFrame(evals).T

,eval_loss,eval_rouge1,eval_rouge2,eval_rougeL,eval_rougeLsum,eval_gen_len,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
UFOAttention,5.628026,17.6677,3.2275,14.1413,14.1415,19.9169,39.4346,10.372,1.750,15.0
MUSEAttention,5.450519,13.6378,2.0293,10.9271,10.9229,19.7433,41.0359,9.967,1.681,15.0
SimplifiedScaledDotProductAttention,5.780069,19.5277,3.9070,16.0524,16.0592,19.7433,38.8754,10.521,1.775,15.0
ScaledDotProductAttention,6.056010,18.6185,3.8945,15.3052,15.2656,19.6406,39.6815,10.307,1.739,15.0
ExternalAttention,5.803121,20.1092,4.1333,16.0511,16.0491,19.8484,38.4355,10.641,1.795,15.0


In [5]:
# 2 attn /w attn
import pandas as pd
pd.DataFrame(evals).T

,eval_loss,eval_rouge1,eval_rouge2,eval_rougeL,eval_rougeLsum,eval_gen_len,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
UFOAttention,5.487316,16.7338,3.0192,13.5056,13.5193,19.8068,40.8118,10.022,1.691,15.0
MUSEAttention,5.240698,13.1814,1.8514,10.6742,10.6683,19.8631,43.8312,9.331,1.574,15.0
SimplifiedScaledDotProductAttention,5.704356,19.8911,4.1914,15.7874,15.8050,19.6479,39.3545,10.393,1.753,15.0
ScaledDotProductAttention,5.917782,18.8718,4.3341,15.3046,15.3024,19.1247,40.2317,10.166,1.715,15.0
ExternalAttention,5.658618,21.2074,4.8898,16.6838,16.7236,19.7824,38.8799,10.520,1.775,15.0


### basic 

In [7]:
from torch import nn
evals_base = {}
for i,attn in enumerate(['base']):

    import os
    model_name = attn
    # model_name = model_checkpoint.split("/")[-1] + model_name
    saved_path = os.path.join('./output',model_name)
    
    args = Seq2SeqTrainingArguments(
        saved_path,
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=15,
        predict_with_generate=True,
        fp16=True,
        save_strategy='epoch',
        load_best_model_at_end=True,
    #     push_to_hub=True,
    )

    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tr,
        eval_dataset=val,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    
    evals[f'{model_name}'] = trainer.evaluate(eval_dataset=te)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 2550
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather a

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,9.611067,0.000000,0.000000,0.000000,0.000000,2.000000
2,No log,9.436999,0.000000,0.000000,0.000000,0.000000,2.000000
3,9.370300,9.270031,0.000000,0.000000,0.000000,0.000000,2.000000
4,9.370300,9.115933,0.000000,0.000000,0.000000,0.000000,2.000000
5,9.370300,8.973625,0.000000,0.000000,0.000000,0.000000,2.000000
6,8.827900,8.845672,0.000000,0.000000,0.000000,0.000000,2.000000
7,8.827900,8.732598,0.000000,0.000000,0.000000,0.000000,2.000000
8,8.827900,8.635551,0.000000,0.000000,0.000000,0.000000,2.000000
9,8.406700,8.551472,0.000000,0.000000,0.000000,0.000000,2.000000
10,8.406700,8.482576,0.000000,0.000000,0.000000,0.000000,2.000000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 6
Saving model checkpoint to ./output/base/checkpoint-170
Configuration saved in ./output/base/checkpoint-170/config.json
Model weights saved in ./output/base/checkpoint-170/pytorch_model.bin
tokenizer config file saved in ./output/base/checkpoint-170/tokenizer_config.json
Special tokens file saved in ./output/base/checkpoint-170/special_tokens_map.json
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/transformers/trainer.py:1357: FutureWarning: Non-finite no